# Parsing necessary information from JSON

In [155]:
import pandas as pd
import json

In [183]:
"""

Travel Chat Bot for Slack
title:  amadeus_query.py
author: George Thoraninh
usage:  python amadeus_query.py

"""
# import statements
import os
import ssl
import requests
from functools import lru_cache

amadeus_api_key = os.environ['AMADEUS_API_KEY']
amadeus_api_secret = os.environ['AMADEUS_API_SECRET']


auth_response = requests.post('https://api.amadeus.com/v1/security/oauth2/token',
                              data={'grant_type': 'client_credentials',
                                    'client_id': amadeus_api_key,
                                    'client_secret': amadeus_api_secret})
bearer = auth_response.json()['access_token']
#print(bearer)
# API has quotas, so better to cache everything
@lru_cache(maxsize=2048)
def call_api(url, **params):
    full_url = f'https://api.amadeus.com/v1{url}'
    response = requests.get(full_url,
                            params=params,
                            headers={'Authorization': f'Bearer {bearer}',
                                     'Content-Type': 'application/vnd.amadeus+json'})
    return response.json()

response = call_api('/shopping/flight-offers',
                origin='YYZ',
                destination='JFK',
                departureDate='2020-08-01',
                returnDate='2020-08-15',
                adults='1',
                nonStop='true',
                currency='CAD',
                maxPrice='8000',
                max='2'
                )
print(response.keys())

dict_keys(['data', 'dictionaries', 'meta'])


In [397]:
# Converts a list to dict; removal is an integer to delimit the square brackets
def list_to_dict(list_to_convert, removal):
    converted_dict = json.loads(str(list_to_convert)[removal:-removal].replace('\'','\"')) 
    return converted_dict

In [493]:
def flight_offer_response(response):

    offer_list = []
    num_offer = 0
    # every offer form the response
    locations = [*response['dictionaries']['locations'].keys()] #gets a list of airport codes

    for offer in response['data']:
        # Converted to string so we can get rid of square brackets [], then replaced by 
        services_dict = list_to_dict(offer['offerItems'],1)
        price = services_dict['price']['total']
        depart_flight_dict = list_to_dict([*services_dict['services'][0].values()], 2)
        arrive_flight_dict = list_to_dict([*services_dict['services'][1].values()], 2)
        num_ticket_avail = depart_flight_dict['pricingDetailPerAdult']['availability']
        # depart time - needs parsing 
        depart_time = depart_flight_dict['flightSegment']['departure']['at']
        arrival_time = depart_flight_dict['flightSegment']['arrival']['at']
        trip_duration = depart_flight_dict['flightSegment']['duration']
        flight_carrier = depart_flight_dict['flightSegment']['carrierCode']
        for key, value in response['dictionaries']['carriers'].items():
            if flight_carrier == key:
                flight_carrier_str = value

        # crafting response for user
        num_offer+=1
        response_str = 'Flight Offer ' + str(num_offer) + ': Toronto - ' + locations[0] + ' -> New York - ' + locations[1] + '\nDeparting at: ' + depart_time + '\nArriving at: ' + arrival_time + '\nTrip Duration: ' + trip_duration + '\nFlight Carrier: ' + flight_carrier_str + '\nPrice: $' + price + ' CAD\n' + str(num_ticket_avail) + ' tickets left!'
        
        offer_list.append(response_str)
        
    return offer_list

In [494]:
offer_list = flight_offer_response(response)

In [496]:
print(*offer_list, sep='\n\n')

Flight Offer 1: Toronto - YYZ -> New York - JFK
Departing at: 2020-08-01T10:15:00-04:00
Arriving at: 2020-08-01T11:49:00-04:00
Trip Duration: 0DT1H34M
Flight Carrier: DELTA AIR LINES
Price: $360.96 CAD
9 tickets left!

Flight Offer 2: Toronto - YYZ -> New York - JFK
Departing at: 2020-08-01T10:15:00-04:00
Arriving at: 2020-08-01T11:49:00-04:00
Trip Duration: 0DT1H34M
Flight Carrier: WESTJET
Price: $427.10 CAD
7 tickets left!
